In [1]:
from collections import defaultdict, Counter
from lxml import etree
from lxml import html
from urllib import error
from urllib import parse
from urllib import request
import time
import json
import os
import re
import socket
import string
import subprocess

In [2]:
POS = ['n'] 
# other combos ['det'], ['prn'], ['cnjsub', 'cnjcoo', 'cnjadv'], ['pr']
POS2 = 'fvfvfvfvfvf'
SECT = 'Numerals'
BIDIX = 'apertium/apertium-ces-rus/apertium-ces-rus.ces-rus.dix'
RUSDIX = 'apertium/apertium-rus/apertium-rus.rus.dix'
CESDIX = 'apertium/apertium-ces/apertium-ces.ces.dix'
GLOSBE = 'https://glosbe.com/ru/cs/'
SEZ = 'https://slovnik.seznam.cz/ru/?q='
LING = 'http://slovniky.lingea.cz/Rusko-cesky/'

In [7]:
def get_paradigms(lang='ru'):
    '''
    gets list of all paradigms with their grammar.
    '''
    pardic = defaultdict(str)
    if lang == 'ru':
        with open(RUSDIX, 'r') as f:
            t = f.readlines()[1:]
    elif lang == 'cs':
        with open(CESDIX, 'r') as f:
            t = f.readlines()[1:]
    skip1 = False
    for line in t:
        line = " ".join(line.split())
        if line.startswith('<pardef n='):
            skip1 = True
            par = line.split('"')[1]
        if skip1 == True and line.startswith('<e') and par != 'numeros':
            parinfo = line.split('<r>')[1].split('</r>')[0]
            # if not parinfo.startswith('<'):
            #     parinfo = parinfo.split('') 
            skip1 = False
            pardic[par] = parinfo
    return pardic


def get_all_pairs(pardic, lang='ru'):
    '''takes POS tag needed, returns all lines with pairs of this POS'''
    words = defaultdict(list)
    if lang == 'ru':
        with open(RUSDIX, 'r') as f:
            t = f.readlines()[1:]
    elif lang == 'cs':
        with open(CESDIX, 'r') as f:
            t = f.readlines()[1:]
    for line in t:
        for ps in POS:
            if (lang == 'ru' and '<e lm=' in line and '_{}"'.format(ps) in line) or\
            (lang == 'cs' and '<e lm=' in line and ('_{}"'.format(ps) in line or '_' + POS2 in line)):
                print(line)
                par = line.split('n="')[1].split('"/></e>')[0]
                lem = line.split('lm="')[1].split('"')[0]
                parinfo = pardic[par]
                parinfo = "<" + "<".join(parinfo.split('<')[1:])
                if ps in ('det', 'prn', 'num'):
                    try:
                        tree = html.fromstring(parinfo)
                        parinfo = "".join([str(html.tostring(x))\
                            .replace('</s>', '').replace('b', '')\
                            .replace('\'', '')
                            for x in tree[0:2]])
                    except: #need to fix tag errors
                        print('TAG ERROR')
                        print(lem)
                        print(parinfo)
                words[lem] = parinfo
    return words



def get_new_translations(rus_words):
    new_trans = defaultdict(list)
    mis_trans = []
    for word, feat in rus_words.items():
        skipg = False
        try:
            new_cur = correct(word, skipg)
        except error.URLError:
            skipg = True
            new_cur = correct(word, skipg)
        print(new_cur)
        if type(new_cur) == str:
            mis_trans.append(new_cur)
        else:
            new_trans[word] += new_cur
        time.sleep(3)
    return new_trans, mis_trans


def correct(lexeme, skipg):
    '''returns a list of lines with pairs'''
    print(lexeme)
    sez_tr = seznam_trans(parse.quote(lexeme.encode()))
    if skipg == False: #checks if in globse, otherwise nothing
        glsb_tr = safe_glosbe_parser(parse.quote(lexeme.encode()))
    else:
        glsb_tr = []
    ling_tr = ling_trans(parse.quote(lexeme.encode()))
    trans = sez_tr + glsb_tr + ling_tr
    trans = clean_list(trans)
    ling_tr = clean_list(ling_tr)

    if not glsb_tr and not sez_tr and len(ling_tr) > 0:
        return ling_tr[0]
    trans2 = [k for k,v in Counter(trans).items() if v>1] #check for multiple instance of same translation.
    if trans2: #if true then return only the common translation.
        return trans2
    elif len(list(set(ling_tr))) in (1, 2):
        return [ling_tr[0]]
    else:
        return lexeme
    # trans = length_cor(lexeme, trans)
    # trans = list(set(trans))
    # print(trans)
    # return trans


def clean_list(trans):
    trans = [x.split(',')[0] if ',' in x else x for x in trans] #takes first word in comma separated list
    trans = [x for x in trans if '.' not in x \
    and x != ' '] #removes entries that include punctuation of empty space
    
    for t in trans:
        if '(' in t or ')' in t: # fix parenthesis issues
            if t.startswith('('):
                test = ' '.join(t.split('(')[1].split(')')[0::1])
                trans.append(test)
            elif t.endswith(')'):
                test = ' '.join(t.split(')')[0].split('(')[0::1])
                trans.append(test)
            trans.remove(t)
            
    trans = [item.rstrip() for item in trans] # removes right hanging whitespace
    for x in trans: #corrects ending mistakes
        for y in trans:
            if x in y and len(x) > len(y):
                trans = trans.remove(y)
    return trans


def length_cor(word, trans):
    w_len = len(word.split(' '))
    for t in trans:
        t_len = len(t.split(' '))
        if t_len > w_len:
            trans.remove(t)
    if trans == None:
        trans = []
    return trans

def safe_glosbe_parser(word):
    page = request.urlopen(GLOSBE + word).read().decode('utf-8')
    translations = html.fromstring(page).xpath('(.//strong[@class=" phr"]/text())[1]')
    print('GLOSBE:{}'.format(translations))
    return translations


def seznam_trans(word):
    page = request.urlopen(SEZ + word).read().decode('utf-8')
    translations = html.fromstring(page).xpath('(.//div[@id="fastMeanings"]/a/text() | .//div[@id="fastMeanings"]/span/text())[1]')
    # translations += html.fromstring(page).xpath('(.//div[@id="fastMeanings"]/a[preceding-sibling::br]/text())[1]')
    print('SEZ{}'.format(translations))
    return translations


def ling_trans(word):
    page = request.urlopen(LING + word).read().decode('utf-8')
    translations = html.fromstring(page).xpath('(.//div[@id="entry-wrapper"]/table/tr[2]/td[2]//span/text())[1]')
    translations += html.fromstring(page).xpath('(.//div[@id="entry-wrapper"]/table/tr[3]/td[2]//span/text())[1]')
    print('LING{}'.format(translations))
    return translations


def combine(trans, rus, cs, mis_trans):
    '''
    combine into bidix lines and append under section.
    '''
    lines = []
    new_lines = []
    for tr_ru, tr_css in trans.items():
        for tr_cs in tr_css:
            if tr_cs not in mis_trans:
                rui = []
                csi = []
                ru_morph = ''.join([x for x in rus[tr_ru]]) #temporary solution to closing tags
                cs_morph = ''.join([x for x in cs[tr_css[0]]])
#                 for i, cr in enumerate(ru_morph):  #closing tags
#                     if cr == '>':
#                         rui.append(i)
#                 for i, cr in enumerate(cs_morph):
#                     if cr == '>':
#                         csi.append(i)
#                 rc = 0
#                 for i in rui:
#                     ru_morph = ru_morph[:i+rc] + '/' + ru_morph[i+rc:]
#                     rc += 1
#                 cc = 0
#                 for i in csi:
#                     cs_morph = cs_morph[:i+cc] + '/' + cs_morph[i+cc:]
#                     cc += 1
                line = '<e><p><l>{0}{1}</l><r>{2}{3}</r></p></e>'.\
                format(tr_cs, cs_morph, tr_ru, ru_morph)
                lines.append(line)
    with open(BIDIX, 'r+') as bix, \
    open('apertium/apertium-ces-rus/apertium-new-ces-rus.ces-rus.dix', 'w') as bixn:
        bilines = bix.readlines()
        for line in bilines:
            if '<!-- Раздел: {}'.format(SECT) in line:
                bixn.write('\t<!-- Раздел: NEW {} -->\n'.format(SECT))
                for x in lines:
                    if not '    {}\n'.format(x) in bilines and not '\t{}\n'.format(x) in bilines:
                        bixn.write('\t{}\n'.format(x))
                    else:
                        print(x)
            bixn.write(line)

In [8]:
pardic = get_paradigms()
wrd_dic = get_all_pairs(pardic)
trans, mis_trans = get_new_translations(wrd_dic)
cs_pardic = get_paradigms(lang='cs')
cs_wrd_dic = get_all_pairs(cs_pardic, lang='cs')


cs_wrds = trans.values() # append more missing values
cs_wrds = [x for y in cs_wrds for x in y]
cs_keys = list(cs_wrd_dic.keys())
for w in cs_wrds:
    if w not in cs_keys:
        mis_trans.append(w)
mis_trans = list(set(mis_trans))


with open('missed/miss_{}.txt'.format(POS), 'w') as miss:
    for x in mis_trans:
        miss.write('{}\n'.format(x))
        
combine(trans, wrd_dic, cs_wrd_dic, mis_trans)

    <!-- ******************************           group n       ****************************** --><e lm="pes"><i></i><par n="pes__n"/></e>

    <e lm="bělogvardějec"><i>bělogvarděj</i><par n="chlap/ec__n"/></e>

    <e lm="Albánec"><i>Albán</i><par n="chlap/ec__n"/></e>

    <e lm="Japonec"><i>Japon</i><par n="chlap/ec__n"/></e>

    <e lm="Kubánec"><i>Kubán</i><par n="chlap/ec__n"/></e>

    <e lm="Makedonec"><i>Makedon</i><par n="chlap/ec__n"/></e>

    <e lm="Portugalec"><i>Portugal</i><par n="chlap/ec__n"/></e>

    <e lm="Vietnamec"><i>Vietnam</i><par n="chlap/ec__n"/></e>

    <e lm="aak"><i>aak</i><par n="bor__n"/></e>

    <e lm="abatyše"><i>abatyš</i><par n="alergi/e__n"/></e>

    <e lm="abdikace"><i>abdikac</i><par n="alergi/e__n"/></e>

    <e lm="abeceda"><i>abeced</i><par n="dob/a__n"/></e>

    <e lm="aberace"><i>aberac</i><par n="alergi/e__n"/></e>

    <e lm="abolice"><i>abolic</i><par n="alergi/e__n"/></e>

    <e lm="abolicionismus"><i>abolicionism</i><par n="ile/us_

    <e lm="radioterapie"><i>radioterapi</i><par n="alergi/e__n"/></e>

    <e lm="recese"><i>reces</i><par n="alergi/e__n"/></e>

    <e lm="reformace"><i>reformac</i><par n="alergi/e__n"/></e>

    <e lm="refrén"><i>refrén</i><par n="arak__n"/></e>

    <e lm="regenerace"><i>regenerac</i><par n="alergi/e__n"/></e>

    <e lm="regent"><i>regent</i><par n="absolvent__n"/></e>

    <e lm="reinkarnace"><i>reinkarnac</i><par n="alergi/e__n"/></e>

    <e lm="remorkér"><i>remorkér</i><par n="arak__n"/></e>

    <e lm="renovace"><i>renovac</i><par n="alergi/e__n"/></e>

    <e lm="reportér"><i>reporté</i><par n="amu/r__n"/></e>

    <e lm="resort"><i>resort</i><par n="arak__n"/></e>

    <e lm="restituce"><i>restituc</i><par n="alergi/e__n"/></e>

    <e lm="revalvace"><i>revalvac</i><par n="alergi/e__n"/></e>

    <e lm="reziduum" r="LR"><p><l>residu</l><r>rezidu</r></p><par n="bary/um__n"/></e>

    <e lm="reziduum"><i>rezidu</i><par n="bary/um__n"/></e>

    <e lm="roháč"><i>roháč</i><par

    <e lm="kokpit"><i>kokpit</i><par n="bor__n"/></e>

    <e lm="koks"><i>koks</i><par n="bor__n"/></e>

    <e lm="koktejl"><i>koktejl</i><par n="arak__n"/></e>

    <e lm="koktejl"><i>koktejl</i><par n="bor__n"/></e>

    <e lm="kolo"><i>kol</i><par n="kol/o__n"/></e>

    <e lm="kolaborace"><i>kolaborac</i><par n="alergi/e__n"/></e>

    <e lm="kolaborant"><i>kolaborant</i><par n="absolvent__n"/></e>

    <e lm="kolchoz"><i>kolchoz</i><par n="bor__n"/></e>

    <e lm="koleda"><i>koled</i><par n="dob/a__n"/></e>

    <e lm="kolej"><i>kolej</i><par n="kolej__n"/></e>

    <e lm="kolekce"><i>kolekc</i><par n="alergi/e__n"/></e>

    <e lm="kolektiv"><i>kolektiv</i><par n="bor__n"/></e>

    <e lm="kolektivizace"><i>kolektivi</i><par n="konver/zace__n"/></e>

    <e lm="kolektivizace"><i>kolektivizac</i><par n="alergi/e__n"/></e>

    <e lm="kolektivizmus"><i>kolektivizm</i><par n="ile/us__n"/></e>

    <e lm="kolibřík"><i>kolibří</i><par n="chalupní/k__n"/></e>

    <e lm="kolize"><i>


    <e lm="radikál"><i>radikál</i><par n="bor__n"/></e>

    <e lm="radioaktivita"><i>radioaktivit</i><par n="dob/a__n"/></e>

    <e lm="radnice"><i>radnic</i><par n="plotic/e__n"/></e>

    <e lm="radon"><i>radon</i><par n="bor__n"/></e>

    <e lm="radost"><i>radost</i><par n="věc__n"/></e>

    <e lm="rafinerie"><i>rafineri</i><par n="alergi/e__n"/></e>

    <e lm="rafinérie"><i>rafinéri</i><par n="alergi/e__n"/></e>

    <e lm="rajonizace"><i>rajonizac</i><par n="alergi/e__n"/></e>

    <e lm="rajón"><i>rajón</i><par n="byt__n"/></e>

    <e lm="raketa"><i>raket</i><par n="dob/a__n"/></e>

    <e lm="raketoplán"><i>raketoplán</i><par n="bor__n"/></e>

    <e lm="rampouch"><i>rampou</i><par n="rampou/ch__n"/></e>

    <e lm="rand"><i>rand</i><par n="arak__n"/></e>

    <e lm="randomizace"><i>randomizac</i><par n="alergi/e__n"/></e>

    <e lm="ranč"><i>ranč</i><par n="keř__n"/></e>

    <e lm="raport"><i>raport</i><par n="arak__n"/></e>

    <e lm="raport"><i>raport</i><par n="bor

    <e lm="folklór"><i>folklór</i><par n="arak__n"/></e>

    <e lm="folklór" r="LR"><p><l>folklor</l><r>folklór</r></p><par n="arak__n"/></e>

    <e lm="fond"><i>fond</i><par n="arak__n"/></e>

    <e lm="foném"><i>foném</i><par n="arak__n"/></e>

    <e lm="forhend"><i>forhend</i><par n="arak__n"/></e>

    <e lm="forint"><i>forint</i><par n="arak__n"/></e>

    <e lm="formaldehyd"><i>formaldehyd</i><par n="arak__n"/></e>

    <e lm="forvard"><i>forvard</i><par n="arak__n"/></e>

    <e lm="fosfát"><i>fosfát</i><par n="arak__n"/></e>

    <e lm="fotoaparát"><i>fotoaparát</i><par n="arak__n"/></e>

    <e lm="foton"><i>foton</i><par n="arak__n"/></e>

    <e lm="foyer"><i>foyer</i><par n="arak__n"/></e>

    <e lm="fór"><i>fór</i><par n="arak__n"/></e>

    <e lm="fragment"><i>fragment</i><par n="arak__n"/></e>

    <e lm="freon"><i>freon</i><par n="arak__n"/></e>

    <e lm="fun"><i>fun</i><par n="arak__n"/></e>

    <e lm="futsal"><i>futsal</i><par n="arak__n"/></e>

    <e lm="gal

...


    <e lm="potupa"><i>potup</i><par n="dob/a__n"/></e>

    <e lm="poživatina"><i>poživatin</i><par n="dob/a__n"/></e>

    <e lm="praktika"><i>praktik</i><par n="dob/a__n"/></e>

    <e lm="prasklina"><i>prasklin</i><par n="dob/a__n"/></e>

    <e lm="pravda"><i>pravd</i><par n="dob/a__n"/></e>

    <e lm="prázdnota"><i>prázdnot</i><par n="dob/a__n"/></e>

    <e lm="prebenda"><i>prebend</i><par n="dob/a__n"/></e>

    <e lm="předmluva"><i>předmluv</i><par n="dob/a__n"/></e>

    <e lm="předpona"><i>předpon</i><par n="dob/a__n"/></e>

    <e lm="představa"><i>představ</i><par n="dob/a__n"/></e>

    <e lm="přehrada"><i>přehrad</i><par n="dob/a__n"/></e>

    <e lm="přeměna"><i>přeměn</i><par n="dob/a__n"/></e>

    <e lm="přeprava"><i>přeprav</i><par n="dob/a__n"/></e>

    <e lm="převýchova"><i>převýchov</i><par n="dob/a__n"/></e>

    <e lm="příčina"><i>příčin</i><par n="dob/a__n"/></e>

    <e lm="příhoda"><i>příhod</i><par n="dob/a__n"/></e>

    <e lm="přímluva"><i>přímluv</i><p

    <e lm="ještěrka"><i>ještěr</i><par n="dív/ka__n"/></e>

    <e lm="jímka"><i>jím</i><par n="dív/ka__n"/></e>

    <e lm="jíška"><i>jíš</i><par n="dív/ka__n"/></e>

    <e lm="jiskřička"><i>jiskřič</i><par n="dív/ka__n"/></e>

    <e lm="jitřenka"><i>jitřen</i><par n="dív/ka__n"/></e>

    <e lm="jízdenka"><i>jízden</i><par n="dív/ka__n"/></e>

    <e lm="judistka"><i>judist</i><par n="dív/ka__n"/></e>

    <e lm="juniorka"><i>junior</i><par n="dív/ka__n"/></e>

    <e lm="kačenka"><i>kačen</i><par n="dív/ka__n"/></e>

    <e lm="kadetka"><i>kadet</i><par n="dív/ka__n"/></e>

    <e lm="kádrovák"><i>kádrovač</i><par n="dív/ka__n"/></e>

    <e lm="kajakářka"><i>kajakář</i><par n="dív/ka__n"/></e>

    <e lm="kamarádka"><i>kamarád</i><par n="dív/ka__n"/></e>

    <e lm="kampelička"><i>kampelič</i><par n="dív/ka__n"/></e>

    <e lm="kandidátka"><i>kandidát</i><par n="dív/ka__n"/></e>

    <e lm="kapička"><i>kapič</i><par n="dív/ka__n"/></e>

    <e lm="kapitánka"><i>kapitán</i><par n


    <e lm="paralela"><i>paralel</i><par n="kos/a__n"/></e>

    <e lm="paralýza"><i>paralýz</i><par n="kos/a__n"/></e>

    <e lm="paralýza" r="LR"><p><l>paralys</l><r>paralýz</r></p><par n="kos/a__n"/></e>

    <e lm="parcela"><i>parcel</i><par n="kos/a__n"/></e>

    <e lm="patrola"><i>patrol</i><par n="kos/a__n"/></e>

    <e lm="pauza"><i>pauz</i><par n="kos/a__n"/></e>

    <e lm="pila"><i>pil</i><par n="kos/a__n"/></e>

    <e lm="píšťala"><i>píšťal</i><par n="kos/a__n"/></e>

    <e lm="pizza"><i>pizz</i><par n="kos/a__n"/></e>

    <e lm="pochvala"><i>pochval</i><par n="kos/a__n"/></e>

    <e lm="posila"><i>posil</i><par n="kos/a__n"/></e>

    <e lm="póza"><i>póz</i><par n="kos/a__n"/></e>

    <e lm="předkapela"><i>předkapel</i><par n="kos/a__n"/></e>

    <e lm="premisa"><i>premis</i><par n="kos/a__n"/></e>

    <e lm="přesila"><i>přesil</i><par n="kos/a__n"/></e>

    <e lm="prognóza"><i>prognóz</i><par n="kos/a__n"/></e>

    <e lm="protéza"><i>protéz</i><par n="kos/a__n


    <e lm="zkušebnictví"><i>zkušebnictví</i><par n="hospodářství__n"/></e>

    <e lm="zlatnictví"><i>zlatnictví</i><par n="hospodářství__n"/></e>

    <e lm="zmatení"><i>změtení</i><par n="hospodářství__n"/></e>

    <e lm="zmrtvýchvstání"><i>zmrtvýchvstání</i><par n="hospodářství__n"/></e>

    <e lm="znamení"><i>znamení</i><par n="hospodářství__n"/></e>

    <e lm="znovuzavedení"><i>znovuzavedení</i><par n="hospodářství__n"/></e>

    <e lm="zoufalství"><i>zoufalství</i><par n="hospodářství__n"/></e>

    <e lm="zpravodajství"><i>zpravodajství</i><par n="hospodářství__n"/></e>

    <e lm="zprostit"><i>sproštění</i><par n="hospodářství__n"/></e>

    <e lm="zrní"><i>zrní</i><par n="hospodářství__n"/></e>

    <e lm="agrárník"><i>agrární</i><par n="chalupní/k__n"/></e>

    <e lm="atentátník"><i>atentátní</i><par n="chalupní/k__n"/></e>

    <e lm="back"><i>bac</i><par n="chalupní/k__n"/></e>

    <e lm="básník"><i>básní</i><par n="chalupní/k__n"/></e>

    <e lm="beatník"><i>beatní<

    <e lm="gesto"><i>gest</i><par n="práv/o__n"/></e>

    <e lm="ghetto"><i>ghett</i><par n="práv/o__n"/></e>

    <e lm="gusto"><i>gust</i><par n="práv/o__n"/></e>

    <e lm="hnojivo"><i>hnojiv</i><par n="práv/o__n"/></e>

    <e lm="inkaso"><i>inkas</i><par n="práv/o__n"/></e>

    <e lm="inkognito"><i>incognit</i><par n="práv/o__n"/></e>

    <e lm="inkognito"><i>inkognit</i><par n="práv/o__n"/></e>

    <e lm="intermezzo"><i>intermezz</i><par n="práv/o__n"/></e>

    <e lm="jamo"><i>jam</i><par n="práv/o__n"/></e>

    <e lm="judo"><i>jud</i><par n="práv/o__n"/></e>

    <e lm="kamenivo"><i>kameniv</i><par n="práv/o__n"/></e>

    <e lm="kasino"><i>kasin</i><par n="práv/o__n"/></e>

    <e lm="kimono"><i>kimon</i><par n="práv/o__n"/></e>

    <e lm="kladivo"><i>kladiv</i><par n="práv/o__n"/></e>

    <e lm="klepeto"><i>klepet</i><par n="práv/o__n"/></e>

    <e lm="komando"><i>komand</i><par n="práv/o__n"/></e>

    <e lm="koryto"><i>koryt</i><par n="práv/o__n"/></e>

    <e lm="

In [217]:
# main()